In [ ]:
!pip install -q -U google-generativeai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 11.6 MB/s eta 0:00:00


In [ ]:
import os
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import random
import time

# --- Agent 1: Info Extractor ---
def agent_extract_info(product_name: str, model: genai.GenerativeModel) -> str:
    """Agent to extract product specifications."""
    print(f"🕵️  Agent Info Extractor activated for: {product_name}")
    prompt = f"""
    You are an Info Extractor agent. Your sole purpose is to find and list the key specifications for a product.
    Your knowledge is current.

    For the product "{product_name}", provide the following details in a clear list format:
    - Price (in INR and USD)
    - Display
    - Camera
    - Battery
    - Processor
    - Any other key features.

    Present the information concisely.
    """
    try:
        config = {"temperature": 0.7 + random.random() * 0.1}
        response = model.generate_content(prompt, generation_config=config)
        return response.text
    except Exception as e:
        print(f"Error in Info Extractor Agent: {e}")
        return f"Could not retrieve information for {product_name}."

# --- Agent 2: Comparator ---
def agent_compare_specs(product_a_name: str, product_b_name: str, product_a_info: str, product_b_info: str, model: genai.GenerativeModel) -> str:
    """Agent to compare specifications and list pros/cons."""
    print(f"⚖️  Agent Comparator activated for {product_a_name} vs. {product_b_name}")
    prompt = f"""
    You are a Comparator agent. You will be given specifications for two products.
    Based ONLY on the information provided below, perform two tasks:
    1. Create a single Markdown table comparing the key specs for both products side-by-side.
    2. Create a "Pros" and "Cons" list for EACH product.

    Do not add any information not present in the input.

    ---
    **Product A: {product_a_name}**
    {product_a_info}
    ---
    **Product B: {product_b_name}**
    {product_b_info}
    ---
    """
    try:
        config = {"temperature": 0.7 + random.random() * 0.1}
        response = model.generate_content(prompt, generation_config=config)
        return response.text
    except Exception as e:
        print(f"Error in Comparator Agent: {e}")
        return "Could not generate comparison."

# --- Agent 3: Recommender ---
def agent_recommend(comparison_data: str, model: genai.GenerativeModel) -> str:
    """Agent to provide a final recommendation."""
    print("🏆 Agent Recommender activated.")
    prompt = f"""
    You are a Recommender agent. You will be given a product comparison.
    Analyze the provided table and pros/cons lists. Your task is to write a concise "Final Verdict".

    In your verdict, recommend one product over the other and clearly explain which type of user would benefit most from that choice.
    Base your verdict strictly on the provided comparison data.

    ---
    **Comparison Data:**
    {comparison_data}
    ---
    """
    try:
        config = {"temperature": 0.7 + random.random() * 0.1}
        response = model.generate_content(prompt, generation_config=config)
        return response.text
    except Exception as e:
        print(f"Error in Recommender Agent: {e}")
        return "Could not generate a final verdict."

# --- Main Orchestrator Function ---
def run_multi_agent_comparison(product_a, product_b):
    """Orchestrates the agent workflow."""
    run_id = random.randint(1000, 9999)
    print(f"\n🚀 Starting Multi-Agent Comparison [Run ID: {run_id}] for {product_a} and {product_b}...")

    if not product_a or not product_b:
        return "Please enter names for both products."

    try:
        # Configure API
        api_key = userdata.get('GEMINI_API_KEY')
        genai.configure(api_key=api_key)

        # Initialize the model using the name from your working reference code
        model = genai.GenerativeModel(
            model_name="gemini-2.5-pro"
        )

        # Step 1: Info Extractor Agent gathers data for both products
        product_a_info = agent_extract_info(product_a, model)
        time.sleep(1) # Small delay to avoid rate limiting issues
        product_b_info = agent_extract_info(product_b, model)

        # Step 2: Comparator Agent creates the comparison
        comparison_output = agent_compare_specs(product_a, product_b, product_a_info, product_b_info, model)

        # Step 3: Recommender Agent provides the verdict
        recommendation_output = agent_recommend(comparison_output, model)

        # Combine the outputs for the final report
        final_report = f"{comparison_output}\n\n---\n\n{recommendation_output}"
        print(f"✅ Comparison Complete [Run ID: {run_id}]")
        return final_report

    except Exception as e:
        print(f"An error occurred in the orchestrator: {e}")
        return f"An unexpected error occurred: {e}"

# --- Gradio UI ---
with gr.Blocks(theme='gradio/soft') as demo:
    gr.Markdown("# 🤖 Multi-Agent Product Comparison Bot")
    gr.Markdown("Enter two product names. Three specialized AI agents will collaborate to generate a detailed comparison report.")

    with gr.Row():
        with gr.Column(scale=4):
            product_a_input = gr.Textbox(label="Product A Name", elem_id="product_a")
            product_b_input = gr.Textbox(label="Product B Name", elem_id="product_b")
            submit_button = gr.Button("Compare Products", variant='primary')
        with gr.Column(scale=1):
            clear_button = gr.ClearButton()

    gr.Examples(
        examples=[
            ["iPhone 16 Pro", "Google Pixel 9 Pro"],
            ["Sony PlayStation 5 Pro", "Xbox Series X"],
            ["Dell XPS 15 (2025)", "Apple MacBook Pro 16 (M4)"],
            ["Tesla Model 3", "Hyundai Ioniq 5"]
        ],
        inputs=[product_a_input, product_b_input]
    )

    output_report = gr.Markdown(label="Comparison Report")

    submit_button.click(
        fn=run_multi_agent_comparison, # Calling the new orchestrator function
        inputs=[product_a_input, product_b_input],
        outputs=output_report,
        api_name="compare_products"
    )

    # The clear button needs to know about all components it should clear
    clear_button.add([product_a_input, product_b_input, output_report])

demo.launch(debug=True, share=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


theme_schema%400.0.3.json: 0.00B [00:00, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0c01ba301a541a5e30.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🚀 Starting Multi-Agent Comparison [Run ID: 9572] for Tesla Model 3 and Hyundai Ioniq 5...
🕵️  Agent Info Extractor activated for: Tesla Model 3
🕵️  Agent Info Extractor activated for: Hyundai Ioniq 5
⚖️  Agent Comparator activated for Tesla Model 3 vs. Hyundai Ioniq 5
🏆 Agent Recommender activated.
✅ Comparison Complete [Run ID: 9572]

🚀 Starting Multi-Agent Comparison [Run ID: 3821] for iphone 16 pro max  and iphone 14 pro max...
🕵️  Agent Info Extractor activated for: iphone 16 pro max 
🕵️  Agent Info Extractor activated for: iphone 14 pro max
⚖️  Agent Comparator activated for iphone 16 pro max  vs. iphone 14 pro max
🏆 Agent Recommender activated.
✅ Comparison Complete [Run ID: 3821]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0c01ba301a541a5e30.gradio.live
